# Evaluate ICL Methods on Selected Datasets

In [5]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from openicl import DatasetReader, PromptTemplate, TopkRetriever, PPLInferencer, AccEvaluator

In [6]:
# from wilds import get_dataset
# from wilds.common.data_loaders import get_train_loader
# dataset = get_dataset(dataset="amazon", download=True)
# display(dataset)

# for record in dataset.get_subset("train"):
#     print(record)
#     break

# train_dict = {
#     "text": [],
#     "label": [],
#     "reviewer_id": []
# }
# for text, label, reviewer_id in dataset.get_subset("train"):
#     train_dict["text"].append(text)
#     train_dict["label"].append(label.item())
#     train_dict["reviewer_id"].append(reviewer_id.tolist())

# test_dict = {
#     "text": [],
#     "label": [],
#     "reviewer_id": []
# }
# for text, label, reviewer_id in dataset.get_subset("test"):
#     test_dict["text"].append(text)
#     test_dict["label"].append(label.item())
#     test_dict["reviewer_id"].append(reviewer_id.tolist())

# full_dataset = DatasetDict()
# full_dataset["train"] = Dataset.from_pandas(pd.DataFrame(train_dict))
# full_dataset["test"] = Dataset.from_pandas(pd.DataFrame(test_dict))
# full_dataset

# display(full_dataset["train"].to_pandas().head())
# display(full_dataset["test"].to_pandas().head())

# full_dataset

In [7]:
# full_dataset["test"].to_pandas().value_counts("label")

In [8]:
# Define a DatasetReader, with specified column names where input and output are stored.
dataset = load_dataset("ag_news")
dataset["train"] = dataset["train"].select(range(100))
dataset["test"] = dataset["test"].select(range(100))
data = DatasetReader(dataset, input_columns=["text"], output_column="label")
tp_dict = {
    0: "</E>World (0) Article: </text>",
    1: "</E>Sports (1) Article: </text>",
    2: "</E>Business (2) Article: </text>",
    3: "</E>Sci/Tech (3) Article: </text>",
}

template = PromptTemplate(tp_dict, {'text': '</text>'}, ice_token='</E>')
# display(template.generate_item(dataset[4590], output_field='label'))
# display(template.generate_item(dataset[6174], output_field='label'))
# display(template.generate_item(dataset[2190], output_field='label'))
# display(template.generate_item(dataset[4983], output_field='label'))

# TopK Retriever
retriever = TopkRetriever(data, ice_num=2, index_split='train', test_split='test')

# # Define a Inferencer
# inferencer = PPLInferencer(model_name='distilgpt2')

# # Inference
# predictions = inferencer.inference(retriever, ice_template=template, output_json_filename='sst2')
# print(predictions)

Found cached dataset ag_news (/home/kyle/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
100%|██████████| 2/2 [00:00<00:00, 27.38it/s]
[2023-04-26 01:31:08,236] [openicl.icl_retriever.icl_topk_retriever] [INFO] Creating index for index set...
100%|██████████| 100/100 [00:02<00:00, 43.30it/s]


In [14]:
sequence = "Calif. Aims to Limit Farm-Related Smog (AP) AP - Southern California's smog-fighting agency went after emissions of the bovine variety Friday, adopting the nation's first rules to reduce air pollution from dairy cow manure."
retriever.tokenizer(sequence, return_tensors="pt")

{'input_ids': tensor([[19619,    13,   317, 12078,   284, 27272, 11272,    12,  9819,  2439,
           519,   357,  2969,     8,  3486,   532,  8050,  3442,   338,   895,
           519,    12, 26594,  4086,  1816,   706,  8971,   286,   262,   275,
           709,   500,  4996,  3217,    11, 22868,   262,  3277,   338,   717,
          3173,   284,  4646,  1633, 12231,   422, 17659,  9875, 48577,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}

In [37]:
input_ids = retriever.tokenizer(sequence, return_tensors="pt")["input_ids"]
example_embedding = retriever.model.encode([sequence], convert_to_numpy=True)
# example_embedding
distances, indices = retriever.index.search(example_embedding, 5)
print(f"distances: {distances}")
print(f"indices: {indices}")

distances: [[0.15052135 0.14935319 0.12454318 0.12422295 0.11615293]]
indices: [[90 12 30 40 24]]


In [32]:
retriever.index.

TypeError: 'Index' object is not subscriptable